<a href="https://colab.research.google.com/github/jansoe/KISchule/blob/main/A6_0_jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. Wortvektoren - Teil 1

## 6.0 Word2Vec: Numerische Repräsentation von Wörten und Wortähnlichkeiten

Die Nachteile von Verfahren wie `TFIDF` liegen auf der Hand: Eine simple Häufigkeitsanalyse von Wörtern in Texten bildet die vielen kontextabhängigen Aspekte von Wörtern nicht mit ab.

Eine Technik, mit der Kontextabhängigkeit automatisiert erlernt werden kann, nennt sich `Word2Vec`. Hierbei wird eine Encoder-Decoder-Architektur verwendet, um ähnlich einem Autoencoder eine abstrakte Repräsentation in Form sogenannter Wortvektoren zu erlernen. Dieser komprimierte Code entspricht dem uns bekannten latenten Raum, wird jedoch im Zusammenhang von Word2Vec als `Embedding` bezeichnet.

Je nach Art des Trainings versucht der Decoder anschließend auf Basis des Embeddings bzw. der Wortvektoren vom Kontext auf einzelne Wörter oder von einzelnen Wörtern auf den Kontext zu schließen.

Im Rahmen des vorliegenden Notebooks A6_0 werden wir uns bereits vorgegebene Embeddings anschauen, also ein bereits trainiertes Modell verwenden. In A6_1 zeigen wir Ihnen, wie Sie anhand von Textdaten ein Word2Vec-Modell tranieren können. Abschließend werfen wir innerhalb von A6_2 bereits einen kleinen Blick auf ein Transformer-Modell, welches auf Basis von Attention-Modulen wesentlich komplexere kontextabhängige Zusammenhänge abbilden kann als ein Word2Vec-Modell. 

Das Verfahren ist wie gehabt:
- Erstellen Sie eine Kopie dieses Notebooks in ihrem Google Drive (vorgeschlagene Umbenennung: "A6_0 - Vorname, Nachname")
- Editieren Sie die Text- und Codezellen.
- Schicken Sie uns einen Freigabelink zum Kommentieren Ihres Notebooks an

In [ ]:
import matplotlib.pyplot as plt
import nltk
import gensim
from pprint import pprint

Die Bibliothek `gensim` ist etwas veraltet und gibt eine Menge Warnungen aus. Wir können diese aber erstmal getrost ignorieren und deren Ausgabe wie folgt verhindern.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Mit dem `gensim.downloader` können wir vortrainierte Wortvektoren herunterladen.

In [ ]:
import gensim.downloader as api
# api.info()

Im Rahmen dieses Notebooks verwenden wir das Modell `glove-wiki-gigaword-300`.
- GloVe ist eine weitere Methode zum Erlernen von Wortvektoren, die zwar nicht auf neuronalen Netzen beruht und daher auch nicht der Kategorie Word2Vec zugeordnet wird. Allerdings liefert GloVe sehr ähnliche Ergebnisse wie die Word2Vec-Varianten CBOW und skip gram.
- Der große Vorteil dieses konkreten Modells ist der zugehörige Trainingsdatensatz, der sowohl die komplette englischsprachige Wikipedia als auch einen umfangreichen Nachrichtenkorpus beinhaltet (Gigaword: https://catalog.ldc.upenn.edu/LDC2011T07), weswegen wir dieses Modell hier verwenden.
- Die Zahl 300 gibt an, dass jedes Wort als ein Vektor der Länge 300 repräsentiert wird.

In [ ]:
model = api.load('glove-wiki-gigaword-300')

### Wortähnlichkeiten

Ein vortrainiertes Word2Vec-Modell kann zu jedem Begriff, der Teil des Korpus ist, diejenigen Wörter aus dem Korpus heraussuchen, die im latenten Raum am nahesten liegen und somit eine verwandte Bedeutung aufweisen:

In [ ]:
model.wv.most_similar('car')

[('cars', 0.7827162742614746),
 ('vehicle', 0.7655367851257324),
 ('truck', 0.7350621819496155),
 ('driver', 0.7114784121513367),
 ('driving', 0.6442224979400635),
 ('vehicles', 0.6328004002571106),
 ('motorcycle', 0.6022512912750244),
 ('automobile', 0.595572829246521),
 ('parked', 0.5910030603408813),
 ('drivers', 0.5778358578681946)]

In [ ]:
model.wv.most_similar('shoe')

[('shoes', 0.7242045998573303),
 ('footwear', 0.5965993404388428),
 ('sneaker', 0.5847950577735901),
 ('sneakers', 0.5435681343078613),
 ('clothing', 0.5319083333015442),
 ('leather', 0.5276093482971191),
 ('apparel', 0.5112749338150024),
 ('boots', 0.4974576234817505),
 ('garment', 0.47937023639678955),
 ('nike', 0.47257572412490845)]

### 6.0.0 Ähnliche Begriffe suchen
Lassen Sie sich zu zwei Begriffen Ihrer Wahl jeweils die drei ähnlichsten Wörter anzeigen. Ist für Sie in beiden Fällen der gemeinsame Kontext aller Wortnachbarn ersichtlich?

---

**Anmerkungen**

Der Parameter `topn` legt die maximale Anzahl an Wörtern fest, die von der Methode `most_similar()` zurückgegeben werden (siehe unten).

### Visualisierung der Wortverwandtschaft mit TSNE
Zuerst visualisieren wir die Verwandtschaftsbeziehungen der Wortvektoren mit TSNE. Dazu lassen wir uns die 2000 Wörter heraussuchen, die dem Wort `intelligence` am nahesten stehen, und erstellen uns eine Liste von deren Vektoren.

In [ ]:
top_similarity = model.wv.most_similar(positive=['computer'], topn=2000)
selected_words = [word for word, similarity in top_similarity]
vectors = [model.wv.word_vec(word, use_norm=True) for word in selected_words]

Jetzt können wir wieder TSNE verwenden, um die hochdimensionalen Vektoren in einem zweidimensionalen Raum zu platzieren und dabei deren lokale Verwandtschaftsbeziehungen beizubehalten.

In [ ]:
import plotly.express as px
import pandas as pd
from sklearn.manifold import TSNE

tsne = TSNE(
    n_components=2,
    perplexity = 5
)
vectors_2D = tsne.fit_transform(vectors)

to_plot = pd.DataFrame(vectors_2D, columns=['x', 'y'])
to_plot['labels'] = selected_words

px.scatter(to_plot, x='x', y='y', hover_name='labels')

Hier kann man noch mehr mit Wort-Einbettungen herumspielen: https://projector.tensorflow.org/

### Analogien im Nachbarschaftsraum

Dank der Vektorrepräsentation eines Word2Vec-Modells lässt sich auch leicht nach analogen Verwandtschaftsbeziehungen suchen. Z.B. können wir folgende Aufgabe stellen:

`berlin` verhält sich zu `germany` wie `???` zu `spain`

Die besten Kandidaten für die Lösung dieser Aufgabe können wir heraussuchen, indem wir uns im Vektorraum zu einem entsprechenden Punkt bewegen (`berlin - germany + spain`) und dort die nahesten Begriffe identifizieren: 

In [ ]:
model.wv.most_similar(positive=['berlin', 'spain'], negative=['germany'], topn=1)

[('madrid', 0.6979373693466187)]

In [ ]:
model.wv.most_similar(positive=['wismar', 'spain'], negative=['germany'], topn=1)

[('cádiz', 0.524493932723999)]

In [ ]:
model.wv.most_similar(positive=['beer', 'italy'], negative=['germany'], topn=1)

[('wine', 0.5321523547172546)]

In [ ]:
model.wv.most_similar(positive=['beer', 'russia'], negative=['germany'], topn=1)

[('vodka', 0.5345780849456787)]

### 6.0.1 Analogien 1
Finden Sie zwei weitere Beispiele für Analogien, die gut funktionieren.

### 6.0.2 Analogien 2
Können Sie zwei Beispiele finden, bei denen dieses Vorgehen nicht funktioniert?

### 6.0.3 Analogien 3
Funktionieren auch Analogien mit mehr als zwei positiven Wortvektoren und/oder mehr als einem negativen Wortvektor?

### Visualisierung der Wortverwandtschaften mit PCA

In [ ]:
words_to_vis = [
    'germany', 'italy', 'france', 'spain', 'uk', 'poland', 'russia', 'austria',
    'berlin', 'rome', 'paris', 'madrid', 'london', 'warsaw', 'moscow', 'vienna', 
]

Wir errechnen zunächst wieder die Wortvektoren der ausgewählten Wörter.

In [ ]:
vectors = [model.wv.word_vec(word, use_norm=True) for word in words_to_vis]

Nun visualisieren wir diese wir in einem 2D-Raum. Anders als zuvor verwenden wir nun jedoch die Methode `PCA`, denn im Gegensatz zu TSNE versucht PCA auch Beziehungen von weit voneinander entfernten Punkten mit abzubilden. Bei wenigen Punkte ist das vorteilhaft, jedoch geht dieser Vorteil bei vielen Punkten schnell verloren, da deren komplexe Nachbarschaftsbeziehungen im hochdimensionalen Raum nicht adäquat in 2D abgebildet werden können.

In [ ]:
from sklearn.decomposition import PCA
import plotly.express as px
import pandas as pd

In [ ]:
pca = PCA(n_components=2)
vectors_2D = pca.fit_transform(vectors)

In [ ]:
to_plot = pd.DataFrame(vectors_2D, columns=['x', 'y'])
to_plot['labels'] = words_to_vis

px.scatter(to_plot, x='x', y='y', hover_name='labels')

### 6.0.4 Visualierung anderer Begriffe
Ändern sie die Begriffe in der Variable `words_to_vis` und führen Sie alle nachfolgenden Zellen nochmals aus. Sind die visuelle Darstellung der Wordvektoren und ihre Nachbarschaftsbeziehungen intuitiv nachvollziehbar? Können Sie Eigenschaften identifizieren, die im Zusammenhang mit der X- und/oder Y-Achse stehen? 

![insitubytes](https://drive.google.com/uc?id=1EAJK7AI9tcZRo3VvYq7vEKGxk7vmK2Ff)